In [1]:
import numpy as np
import pandas as pd
from fmskill import ModelResult, PointObservation, TrackObservation, Connector
#from fmskill.model.xarray import XArrayModelResult, XArrayModelResultItem
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Observations

In [2]:
o1 = PointObservation('../tests/testdata/SW/HKNA_Hm0.dfs0', item=0, x=4.2420, y=52.6887, name="HKNA")
o2 = PointObservation("../tests/testdata/SW/eur_Hm0.dfs0", item=0, x=3.2760, y=51.9990, name="EPL")
o3 = TrackObservation("../tests/testdata/SW/Alti_c2_Dutch.dfs0", item=3, name="c2")

## MIKE ModelResult

In [3]:
mr1 = ModelResult('../tests/testdata/SW/HKZN_local_2017_DutchCoast.dfsu', name='SW_1', item=0)

## NetCDF ModelResult

In [4]:
fn = r"..\tests\testdata\SW\ERA5_DutchCoast.nc"

In [5]:
mr = ModelResult(fn)

In [6]:
mr

<XArrayModelResult> 'ERA5_DutchCoast'
- Item: 0: mwd
- Item: 1: mwp
- Item: 2: mp2
- Item: 3: pp1d
- Item: 4: swh

In [7]:
mr.ds  # mr contains the xr.Dataset

<xarray.Dataset>
Dimensions:  (time: 67, x: 20, y: 11)
Coordinates:
  * x        (x) float32 -1.0 -0.5 0.0 0.5 1.0 1.5 ... 6.0 6.5 7.0 7.5 8.0 8.5
  * y        (y) float32 55.0 54.5 54.0 53.5 53.0 52.5 52.0 51.5 51.0 50.5 50.0
  * time     (time) datetime64[ns] 2017-10-27 ... 2017-10-29T18:00:00
Data variables:
    mwd      (time, y, x) float32 ...
    mwp      (time, y, x) float32 ...
    mp2      (time, y, x) float32 ...
    pp1d     (time, y, x) float32 ...
    swh      (time, y, x) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-06-07 12:25:02 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

## ModelResultItem
We can select an item from the NetCDF file be using square brackets refering either to the item name or number. 

In [8]:
mr['swh']

<XArrayModelResultItem> 'ERA5_DutchCoast'
- Item: swh

## Test extract from XArray

In [9]:
mr['swh'].extract_observation(o1)

NotImplementedError: 

## Connect

In [ ]:
# con = Connector([o1,o2,o3],mr)